In [81]:
import pandas as pd
import numpy as np 
import csv
import re # for regular expression
import string
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from collections import Counter 

%matplotlib inline

In [82]:
Data = pd.read_excel ('EatmarnaDataset.xlsx')

In [83]:
Data.shape

(27870, 7)

In [84]:
Data.columns

Index(['id', '@username', 'display name', 'text', 'date', 'retweets', 'likes'], dtype='object')

In [85]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27870 entries, 0 to 27869
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            27870 non-null  int64 
 1   @username     27870 non-null  object
 2   display name  27865 non-null  object
 3   text          27870 non-null  object
 4   date          27870 non-null  object
 5   retweets      27870 non-null  int64 
 6   likes         27870 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 1.5+ MB


In [86]:
Data.drop_duplicates(subset = "text", keep = False, inplace = True) 
Data.head(20)

,id,@username,display name,text,date,retweets,likes
0,1456153547188445185,jiil3r,حجز العمرة #اعتمرنا,#اعتمرنا اصدار تصريح 🛑.\nمتوفر حجز تصاريح \nالعمرة📍\nالصلاة في المسجد الحرام📍\nالصلاة في الروضة الشريفة )رجال+نساء📍\nزيارة النبي وصاحبيه رضوان الله عليهما📍.\nللحجوزات تصريح ليوم الخميس والجمعة القادم تواصل معي عبر الواتس 0577805768🕋✅.,2021-11-04T06:57:14+00:00,0,0
2,1456151700822241284,eatmarna14,حجز تصاريح عمره #اعتمرنا,من حجوزاتي الان صلاة الروضة الشريفة ليوم الجمعة للرجال والنساء تقبل الله منهم صالح الاعمال لمن يرغب الحجز تواصل معي عبر الواتس 0577805768✅ #اعتمرنا https://t.co/Sc6suae0R2,2021-11-04T06:49:54+00:00,0,0
3,1456149271984680960,jiil3r,حجز العمرة #اعتمرنا,اصدار تصريح 🟢.\nنستخرج لكم تصاريح من تطبيق اعتمرنا \nتصريح صلاة الروضة الشريفة للرجال والنساء لجميع الايام في #المسجد_النبوي_الشريف #المدينة__المنورة \nلمن يرغب الحجز تواصل معي عبر الواتس 0577805768✅ #اعتمرنا https://t.co/MswcJ9S8qf,2021-11-04T06:40:15+00:00,0,0
4,1456147951710052354,9__Rema,ريما,#الاتحاد_الفتح\nاللي يبي حجز عمره أو صلاه أو زياره اليوم الخميس أو بكرى الجمعه يتواصل خاص✔ #اعتمرنا,2021-11-04T06:35:00+00:00,0,0
5,1456147896798167044,9__Rema,ريما,#كفر_قوي\nاللي يبي حجز عمره أو صلاه أو زياره اليوم الخميس أو بكرى الجمعه يتواصل خاص✔ #اعتمرنا,2021-11-04T06:34:47+00:00,0,0
6,1456147812844986368,9__Rema,ريما,#الخميس_الونيس\nاللي يبي حجز عمره أو صلاه أو زياره اليوم الخميس أو بكرى الجمعه يتواصل خاص✔ #اعتمرنا.,2021-11-04T06:34:27+00:00,0,0
7,1456147731811012616,9__Rema,ريما,#صباح_الخير\nاللي يبي حجز عمره أو صلاه أو زياره اليوم الخميس أو بكرى الجمعه يتواصل خاص✔ #اعتمرنا.,2021-11-04T06:34:07+00:00,0,0
8,1456133909658456067,jiil3r,حجز العمرة #اعتمرنا,اصدار تصريح ♻️.\nنستخرج لكم تصاريح من تطبيق اعتمرنا \nتصريح صلاة الجمعة وجميع صلوات \nوصلاة الروضة الشريفة \nوحجز عمرة \nوزيارة النبي وصاحبيه لمن يرغب الحجز تواصل معي عبر الواتس 0577805768✅ #الجمعه #يوم_الجمعة #اعتمرنا,2021-11-04T05:39:12+00:00,0,0
9,1456133550156263426,hajaz0580,حجز عمره #اعتمرنا,اصدار تصريح ♻️.\nنستخرج لكم تصاريح من تطبيق اعتمرنا \nتصريح صلاة الجمعة وجميع صلوات \nوصلاة الروضة الشريفة \nوحجز عمرة \nوزيارة النبي وصاحبيه لمن يرغب الحجز تواصل معي عبر الواتس 0578403597✅ #اعتمرنا,2021-11-04T05:37:46+00:00,0,0
10,1456117794437742599,3mrah12,رحآب | خدمات الكترونيه 🤍.,#اعتمرنا متوفر حجز تصريح الروضة الشريفة لمن يرغب الحجز تواصل معي عبر الواتس اب 0559932482🌱📱#توكلنا #صلاة_الاستسقاء ،,2021-11-04T04:35:10+00:00,0,0


In [87]:
Data.shape

(24632, 7)

In [88]:
# Removing mentions and links from tweets
Data['text']= Data['text'].str.replace("(@[A-Za-z0-9_-]+)|(#[A-Za-z0-9_-]+)|http\S+", "")

<ipython-input-88-d082835e96a7>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  Data['text']= Data['text'].str.replace("(@[A-Za-z0-9_-]+)|(#[A-Za-z0-9_-]+)|http\S+", "")


In [89]:
# Removing hashtags and new lines from tweets
def processPost(tweet):
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    
    #Remove new lines with white space
    tweet = tweet.replace('\n',' ')
    tweet = tweet.replace('\t',' ')
    tweet = tweet.replace('_',' ')
    tweet = tweet.replace('\r',' ')
        
    return tweet

In [90]:
Data["text"] = Data['text'].apply(lambda x: processPost(x))

In [91]:
Data.head(20)

,id,@username,display name,text,date,retweets,likes
0,1456153547188445185,jiil3r,حجز العمرة #اعتمرنا,اعتمرنا اصدار تصريح 🛑. متوفر حجز تصاريح العمرة📍 الصلاة في المسجد الحرام📍 الصلاة في الروضة الشريفة )رجال+نساء📍 زيارة النبي وصاحبيه رضوان الله عليهما📍. للحجوزات تصريح ليوم الخميس والجمعة القادم تواصل معي عبر الواتس 0577805768🕋✅.,2021-11-04T06:57:14+00:00,0,0
2,1456151700822241284,eatmarna14,حجز تصاريح عمره #اعتمرنا,من حجوزاتي الان صلاة الروضة الشريفة ليوم الجمعة للرجال والنساء تقبل الله منهم صالح الاعمال لمن يرغب الحجز تواصل معي عبر الواتس 0577805768✅ اعتمرنا,2021-11-04T06:49:54+00:00,0,0
3,1456149271984680960,jiil3r,حجز العمرة #اعتمرنا,اصدار تصريح 🟢. نستخرج لكم تصاريح من تطبيق اعتمرنا تصريح صلاة الروضة الشريفة للرجال والنساء لجميع الايام في المسجد النبوي الشريف المدينة المنورة لمن يرغب الحجز تواصل معي عبر الواتس 0577805768✅ اعتمرنا,2021-11-04T06:40:15+00:00,0,0
4,1456147951710052354,9__Rema,ريما,الاتحاد الفتح اللي يبي حجز عمره أو صلاه أو زياره اليوم الخميس أو بكرى الجمعه يتواصل خاص✔ اعتمرنا,2021-11-04T06:35:00+00:00,0,0
5,1456147896798167044,9__Rema,ريما,كفر قوي اللي يبي حجز عمره أو صلاه أو زياره اليوم الخميس أو بكرى الجمعه يتواصل خاص✔ اعتمرنا,2021-11-04T06:34:47+00:00,0,0
6,1456147812844986368,9__Rema,ريما,الخميس الونيس اللي يبي حجز عمره أو صلاه أو زياره اليوم الخميس أو بكرى الجمعه يتواصل خاص✔ اعتمرنا.,2021-11-04T06:34:27+00:00,0,0
7,1456147731811012616,9__Rema,ريما,صباح الخير اللي يبي حجز عمره أو صلاه أو زياره اليوم الخميس أو بكرى الجمعه يتواصل خاص✔ اعتمرنا.,2021-11-04T06:34:07+00:00,0,0
8,1456133909658456067,jiil3r,حجز العمرة #اعتمرنا,اصدار تصريح ♻️. نستخرج لكم تصاريح من تطبيق اعتمرنا تصريح صلاة الجمعة وجميع صلوات وصلاة الروضة الشريفة وحجز عمرة وزيارة النبي وصاحبيه لمن يرغب الحجز تواصل معي عبر الواتس 0577805768✅ الجمعه يوم الجمعة اعتمرنا,2021-11-04T05:39:12+00:00,0,0
9,1456133550156263426,hajaz0580,حجز عمره #اعتمرنا,اصدار تصريح ♻️. نستخرج لكم تصاريح من تطبيق اعتمرنا تصريح صلاة الجمعة وجميع صلوات وصلاة الروضة الشريفة وحجز عمرة وزيارة النبي وصاحبيه لمن يرغب الحجز تواصل معي عبر الواتس 0578403597✅ اعتمرنا,2021-11-04T05:37:46+00:00,0,0
10,1456117794437742599,3mrah12,رحآب | خدمات الكترونيه 🤍.,اعتمرنا متوفر حجز تصريح الروضة الشريفة لمن يرغب الحجز تواصل معي عبر الواتس اب 0559932482🌱📱توكلنا صلاة الاستسقاء ،,2021-11-04T04:35:10+00:00,0,0


In [92]:
# remove punctuations 
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [93]:
Data["text"] = Data['text'].apply(lambda x: remove_punctuations(x))

In [94]:
Data.head(20)

,id,@username,display name,text,date,retweets,likes
0,1456153547188445185,jiil3r,حجز العمرة #اعتمرنا,اعتمرنا اصدار تصريح 🛑 متوفر حجز تصاريح العمرة📍 الصلاة في المسجد الحرام📍 الصلاة في الروضة الشريفة رجالنساء📍 زيارة النبي وصاحبيه رضوان الله عليهما📍 للحجوزات تصريح ليوم الخميس والجمعة القادم تواصل معي عبر الواتس 0577805768🕋✅,2021-11-04T06:57:14+00:00,0,0
2,1456151700822241284,eatmarna14,حجز تصاريح عمره #اعتمرنا,من حجوزاتي الان صلاة الروضة الشريفة ليوم الجمعة للرجال والنساء تقبل الله منهم صالح الاعمال لمن يرغب الحجز تواصل معي عبر الواتس 0577805768✅ اعتمرنا,2021-11-04T06:49:54+00:00,0,0
3,1456149271984680960,jiil3r,حجز العمرة #اعتمرنا,اصدار تصريح 🟢 نستخرج لكم تصاريح من تطبيق اعتمرنا تصريح صلاة الروضة الشريفة للرجال والنساء لجميع الايام في المسجد النبوي الشريف المدينة المنورة لمن يرغب الحجز تواصل معي عبر الواتس 0577805768✅ اعتمرنا,2021-11-04T06:40:15+00:00,0,0
4,1456147951710052354,9__Rema,ريما,الاتحاد الفتح اللي يبي حجز عمره أو صلاه أو زياره اليوم الخميس أو بكرى الجمعه يتواصل خاص✔ اعتمرنا,2021-11-04T06:35:00+00:00,0,0
5,1456147896798167044,9__Rema,ريما,كفر قوي اللي يبي حجز عمره أو صلاه أو زياره اليوم الخميس أو بكرى الجمعه يتواصل خاص✔ اعتمرنا,2021-11-04T06:34:47+00:00,0,0
6,1456147812844986368,9__Rema,ريما,الخميس الونيس اللي يبي حجز عمره أو صلاه أو زياره اليوم الخميس أو بكرى الجمعه يتواصل خاص✔ اعتمرنا,2021-11-04T06:34:27+00:00,0,0
7,1456147731811012616,9__Rema,ريما,صباح الخير اللي يبي حجز عمره أو صلاه أو زياره اليوم الخميس أو بكرى الجمعه يتواصل خاص✔ اعتمرنا,2021-11-04T06:34:07+00:00,0,0
8,1456133909658456067,jiil3r,حجز العمرة #اعتمرنا,اصدار تصريح ♻️ نستخرج لكم تصاريح من تطبيق اعتمرنا تصريح صلاة الجمعة وجميع صلوات وصلاة الروضة الشريفة وحجز عمرة وزيارة النبي وصاحبيه لمن يرغب الحجز تواصل معي عبر الواتس 0577805768✅ الجمعه يوم الجمعة اعتمرنا,2021-11-04T05:39:12+00:00,0,0
9,1456133550156263426,hajaz0580,حجز عمره #اعتمرنا,اصدار تصريح ♻️ نستخرج لكم تصاريح من تطبيق اعتمرنا تصريح صلاة الجمعة وجميع صلوات وصلاة الروضة الشريفة وحجز عمرة وزيارة النبي وصاحبيه لمن يرغب الحجز تواصل معي عبر الواتس 0578403597✅ اعتمرنا,2021-11-04T05:37:46+00:00,0,0
10,1456117794437742599,3mrah12,رحآب | خدمات الكترونيه 🤍.,اعتمرنا متوفر حجز تصريح الروضة الشريفة لمن يرغب الحجز تواصل معي عبر الواتس اب 0559932482🌱📱توكلنا صلاة الاستسقاء,2021-11-04T04:35:10+00:00,0,0


In [103]:

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
ar_stops = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/macbook/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [104]:

for word in Data['text']:
    if word in ar_stops:        
        words_filtered.remove(word)

In [105]:
Data.shape

(24632, 8)

In [98]:
Data.dropna(subset = ["text"], inplace=True)

In [99]:
tokenizer = RegexpTokenizer(r'\w+')
Data["tokens"] = Data["text"].apply(tokenizer.tokenize)

In [100]:
all_words = [word for tokens in Data["tokens"] for word in tokens]
sentence_lengths = [len(tokens) for tokens in Data["tokens"]]

VOCAB = sorted(list(set(all_words)))

print("%s words total, with a vocabulary size of %s" % (len(all_words), len(VOCAB)))
print("Max sentence length is %s" % max(sentence_lengths))

508018 words total, with a vocabulary size of 25557
Max sentence length is 95


In [101]:
Data.head(20)

,id,@username,display name,text,date,retweets,likes,tokens
0,1456153547188445185,jiil3r,حجز العمرة #اعتمرنا,اعتمرنا اصدار تصريح 🛑 متوفر حجز تصاريح العمرة📍 الصلاة في المسجد الحرام📍 الصلاة في الروضة الشريفة رجالنساء📍 زيارة النبي وصاحبيه رضوان الله عليهما📍 للحجوزات تصريح ليوم الخميس والجمعة القادم تواصل معي عبر الواتس 0577805768🕋✅,2021-11-04T06:57:14+00:00,0,0,"[اعتمرنا, اصدار, تصريح, متوفر, حجز, تصاريح, العمرة, الصلاة, في, المسجد, الحرام, الصلاة, في, الروضة, الشريفة, رجالنساء, زيارة, النبي, وصاحبيه, رضوان, الله, عليهما, للحجوزات, تصريح, ليوم, الخميس, والجمعة, القادم, تواصل, معي, عبر, الواتس, 0577805768]"
2,1456151700822241284,eatmarna14,حجز تصاريح عمره #اعتمرنا,من حجوزاتي الان صلاة الروضة الشريفة ليوم الجمعة للرجال والنساء تقبل الله منهم صالح الاعمال لمن يرغب الحجز تواصل معي عبر الواتس 0577805768✅ اعتمرنا,2021-11-04T06:49:54+00:00,0,0,"[من, حجوزاتي, الان, صلاة, الروضة, الشريفة, ليوم, الجمعة, للرجال, والنساء, تقبل, الله, منهم, صالح, الاعمال, لمن, يرغب, الحجز, تواصل, معي, عبر, الواتس, 0577805768, اعتمرنا]"
3,1456149271984680960,jiil3r,حجز العمرة #اعتمرنا,اصدار تصريح 🟢 نستخرج لكم تصاريح من تطبيق اعتمرنا تصريح صلاة الروضة الشريفة للرجال والنساء لجميع الايام في المسجد النبوي الشريف المدينة المنورة لمن يرغب الحجز تواصل معي عبر الواتس 0577805768✅ اعتمرنا,2021-11-04T06:40:15+00:00,0,0,"[اصدار, تصريح, نستخرج, لكم, تصاريح, من, تطبيق, اعتمرنا, تصريح, صلاة, الروضة, الشريفة, للرجال, والنساء, لجميع, الايام, في, المسجد, النبوي, الشريف, المدينة, المنورة, لمن, يرغب, الحجز, تواصل, معي, عبر, الواتس, 0577805768, اعتمرنا]"
4,1456147951710052354,9__Rema,ريما,الاتحاد الفتح اللي يبي حجز عمره أو صلاه أو زياره اليوم الخميس أو بكرى الجمعه يتواصل خاص✔ اعتمرنا,2021-11-04T06:35:00+00:00,0,0,"[الاتحاد, الفتح, اللي, يبي, حجز, عمره, أو, صلاه, أو, زياره, اليوم, الخميس, أو, بكرى, الجمعه, يتواصل, خاص, اعتمرنا]"
5,1456147896798167044,9__Rema,ريما,كفر قوي اللي يبي حجز عمره أو صلاه أو زياره اليوم الخميس أو بكرى الجمعه يتواصل خاص✔ اعتمرنا,2021-11-04T06:34:47+00:00,0,0,"[كفر, قوي, اللي, يبي, حجز, عمره, أو, صلاه, أو, زياره, اليوم, الخميس, أو, بكرى, الجمعه, يتواصل, خاص, اعتمرنا]"
6,1456147812844986368,9__Rema,ريما,الخميس الونيس اللي يبي حجز عمره أو صلاه أو زياره اليوم الخميس أو بكرى الجمعه يتواصل خاص✔ اعتمرنا,2021-11-04T06:34:27+00:00,0,0,"[الخميس, الونيس, اللي, يبي, حجز, عمره, أو, صلاه, أو, زياره, اليوم, الخميس, أو, بكرى, الجمعه, يتواصل, خاص, اعتمرنا]"
7,1456147731811012616,9__Rema,ريما,صباح الخير اللي يبي حجز عمره أو صلاه أو زياره اليوم الخميس أو بكرى الجمعه يتواصل خاص✔ اعتمرنا,2021-11-04T06:34:07+00:00,0,0,"[صباح, الخير, اللي, يبي, حجز, عمره, أو, صلاه, أو, زياره, اليوم, الخميس, أو, بكرى, الجمعه, يتواصل, خاص, اعتمرنا]"
8,1456133909658456067,jiil3r,حجز العمرة #اعتمرنا,اصدار تصريح ♻️ نستخرج لكم تصاريح من تطبيق اعتمرنا تصريح صلاة الجمعة وجميع صلوات وصلاة الروضة الشريفة وحجز عمرة وزيارة النبي وصاحبيه لمن يرغب الحجز تواصل معي عبر الواتس 0577805768✅ الجمعه يوم الجمعة اعتمرنا,2021-11-04T05:39:12+00:00,0,0,"[اصدار, تصريح, نستخرج, لكم, تصاريح, من, تطبيق, اعتمرنا, تصريح, صلاة, الجمعة, وجميع, صلوات, وصلاة, الروضة, الشريفة, وحجز, عمرة, وزيارة, النبي, وصاحبيه, لمن, يرغب, الحجز, تواصل, معي, عبر, الواتس, 0577805768, الجمعه, يوم, الجمعة, اعتمرنا]"
9,1456133550156263426,hajaz0580,حجز عمره #اعتمرنا,اصدار تصريح ♻️ نستخرج لكم تصاريح من تطبيق اعتمرنا تصريح صلاة الجمعة وجميع صلوات وصلاة الروضة الشريفة وحجز عمرة وزيارة النبي وصاحبيه لمن يرغب الحجز تواصل معي عبر الواتس 0578403597✅ اعتمرنا,2021-11-04T05:37:46+00:00,0,0,"[اصدار, تصريح, نستخرج, لكم, تصاريح, من, تطبيق, اعتمرنا, تصريح, صلاة, الجمعة, وجميع, صلوات, وصلاة, الروضة, الشريفة, وحجز, عمرة, وزيارة, النبي, وصاحبيه, لمن, يرغب, الحجز, تواصل, معي, عبر, الواتس, 0578403597, اعتمرنا]"
10,1456117794437742599,3mrah12,رحآب | خدمات الكترونيه 🤍.,اعتمرنا متوفر حجز تصريح الروضة الشريفة لمن يرغب الحجز تواصل معي عبر الواتس اب 0559932482🌱📱توكلنا صلاة الاستسقاء,2021-11-04T04:35:10+00:00,0,0,"[اعتمرنا, متوفر, حجز, تصريح, الروضة, الشريفة, لمن, يرغب, الحجز, تواصل, معي, عبر, الواتس, اب, 0559932482, توكلنا, صلاة, الاستسقاء]"


In [106]:
Data.shape

(24632, 8)

In [107]:
Data.columns

Index(['id', '@username', 'display name', 'text', 'date', 'retweets', 'likes',
       'tokens'],
      dtype='object')